### Velocity near boundary

This function computes the velocity of a particle in proximity of a solid (free-slip) boundary (e.g. wall/land).

At the boundary, the interpolant overshoots and extrapolates into a region where the velocity field is not defined anymore. We refer to the area between the last known data point and the boundary as 'boundary layer'. 

Especially "cubic" interpolation methods lead to strong overshooting at the boundaries thereby violating the no-slip boundary condition and the incompressibility assumption of the flow. Hence, computing the velocity at the boundary leads to unphysical lagrangian particle trajectories which violate boundary conditions (Particles cannot penetrate the wall or hit the land).

To evaluate the velocity of the particle on the boundary we must therefore resort to other techniques which assume incompressibility of the flow and no-slip boundary conditions.
We start by introducing coordinates $ \mathbf{x} = (x, y) $ in which the wall aligns with the x axis, and hence the continuously differentiable fluid velocity field $ \mathbf{v}(x, y, t) $ is given by:

\begin{equation}
\mathbf{v}(x, y, t) = \begin{pmatrix} u(x, 0, t) +O(y) \\ \dfrac{1}{2}y^2 v_{yy}(x, 0, t) + O(y^3) \end{pmatrix}
\end{equation}

The distance between the last point where the flow field is defined and the boundary is denoted as $ \delta_{boundary} $ and denotes the thickness of the boundary layer.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| idx_x | float | index which specifies where the particle is currently located in the X-meshgrid: <br /> X[idx_y, idx_x] <= x[0] < X[idx_y, idx_x+1]|
| idx_x | float | index which specifies where the particle is currently located in the Y-meshgrid: <br /> Y[idx_y, idx_x] <= x[1] < X[idx_y+1, idx_x]|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Ny, Nx) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| vx | float | $ u((\mathbf{x}, t)) $ |
| vy | float | $ v((\mathbf{x}, t)) $ |

In [ ]:
# Import numpy
import numpy as np

# Import math tools
from math import sqrt

# Import function that calculates velocity in domain
from ipynb.fs.defs.velocity_domain import velocity_domain

In [ ]:
def velocity_boundary(t, x, idx_x, idx_y, Interpolant, defined_domain, X, Y, bool_unsteady, dt_data, dx_data, dy_data):
    
    check_idx = (1 < idx_x < defined_domain.shape[1] - 1) and (1 < idx_y < defined_domain.shape[0] - 1)
        
    x_p = x[0]
    y_p = x[1]
        
    left_boundary = np.sum(defined_domain[idx_y-1:idx_y+1, idx_x]) == 2 and np.sum(defined_domain[idx_y-1:idx_y+1, idx_x-1]) == 0
    right_boundary = np.sum(defined_domain[idx_y-1:idx_y+1, idx_x]) == 0 and np.sum(defined_domain[idx_y-1:idx_y+1, idx_x-1]) == 2
    upper_boundary = np.sum(defined_domain[idx_y-1,idx_x-1:idx_x+1]) == 2 and np.sum(defined_domain[idx_y,idx_x-1:idx_x+1]) == 0
    lower_boundary = np.sum(defined_domain[idx_y-1,idx_x-1:idx_x+1]) == 0 and np.sum(defined_domain[idx_y,idx_x-1:idx_x+1]) == 2 
        
    # left/right boundary 
    if left_boundary or right_boundary:
            
        if left_boundary: 
                
            # set idx of wall
            idx_wall = idx_x-1
            idx_boundary = idx_wall+1
            
        elif right_boundary:
                
            idx_wall = idx_x
            idx_boundary = idx_wall-1
    
        # x coordinate of the wall
        x_wall = X[0, idx_wall]
                
        # closest point to the wall where the velocity field is still defined (=boundary layer)
        x_boundary_layer = np.array([X[0, idx_boundary], x[1]])
            
        # velocity at the x_boundary_layer at time t
        vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
        # x-distance of particle to wall
        x_tilde = abs(x[0]-x_wall)
              
        # compute velocity from eq. (3)
        u = vel[0]*(x_tilde/dx_data)**2
        v = vel[1]
            
    # upper/lower boundary
    elif upper_boundary or lower_boundary:
            
        if upper_boundary:
                
            idx_wall = idx_y
            idx_boundary = idx_wall-1
                
        elif lower_boundary:
                
            idx_wall = idx_y-1
            idx_boundary = idx_wall+1
        
        # y coordinate of the wall
        y_wall = Y[idx_y, 0]
            
        # closest point to the wall where the velocity field is still defined (=boundary layer)
        x_boundary_layer = np.array([x[0], Y[idx_y-1, 0]])
            
        # velocity at the x_boundary_layer at time t
        vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
                                  
        # y-distance of particle to wall
        y_tilde = abs(y_wall-x[1])
                    
        # compute velocity from eq. (3)
        u = vel[0]
        v = vel[1]*(y_tilde/dy_data)**2
            
    elif np.sum(defined_domain[idx_y-1:idx_y+1, idx_x-1:idx_x+1]) == 1:
            
        if defined_domain[idx_y-1, idx_x-1] or defined_domain[idx_y, idx_x]:
            
            wall_orientation = np.array([-1, 1])
            
            thickness = np.sqrt(dx_data**2+dy_data**2)/2
            
            y_0 = Y[idx_y, idx_x-1]
            x_0 = X[idx_y, idx_x-1]
            
            x_wall = ((y_0-y_p)+x_p+x_0)/2
            y_wall = ((y_0+y_p)+(x_0-x_p))/2
                
            if defined_domain[idx_y-1, idx_x-1]:
            
                x_boundary_layer = np.array([x_wall - thickness/sqrt(2), y_wall - thickness/sqrt(2)])
            
            elif defined_domain[idx_y, idx_x]:
                    
                x_boundary_layer = np.array([x_wall + thickness/sqrt(2), y_wall + thickness/sqrt(2)])
                
            vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
            # Coordinate transformation
            
            y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
            u_new = (-vel[0]+vel[1])/sqrt(2)
            v_new = (-vel[0]-vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
            u = -(u_new+v_new)/sqrt(2)
            v = (u_new-v_new)/sqrt(2)
                
        elif defined_domain[idx_y, idx_x-1] or defined_domain[idx_y - 1, idx_x]:
            
            wall_orientation = np.array([1, 1])
            
            thickness = sqrt(dx_data**2+dy_data**2)/2
            
            y_0 = Y[idx_y-1, idx_x-1]
            x_0 = X[idx_y-1, idx_x-1]
            
            x_wall = ((y_0-y_p)+x_p+x_0)/2
            y_wall = ((y_0+y_p)+(x_0-x_p))/2
                
            if defined_domain[idx_y, idx_x-1]:
            
                x_boundary_layer = np.array([x_wall - thickness/sqrt(2), y_wall + thickness/sqrt(2)])
            
            elif defined_domain[idx_y-1, idx_x]:
                    
                x_boundary_layer = np.array([x_wall + thickness/sqrt(2), y_wall - thickness/sqrt(2)])
            
            vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
            # Coordinate transformation
            
            y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
            u_new = (vel[0]+vel[1])/sqrt(2)
            v_new = (-vel[0]+vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
            u = (u_new-v_new)/sqrt(2)
            v = (u_new+v_new)/sqrt(2)                   
        
    elif np.sum(defined_domain[idx_y-1:idx_y+1, idx_x-1:idx_x+1]) == 3 and check_idx:
        
        if check_idx:
                
            if defined_domain[idx_y, idx_x] == 0:
                    
                x_0 = X[idx_y, idx_x]
                y_0 = Y[idx_y, idx_x]
            
                p1norm = (abs(y_p-y_0)+ abs(x_p-x_0)) < 1
            
                if p1norm and (((y_p-y_0)/(x_p-x_0) <= 1 and defined_domain[idx_y+1, idx_x-1] == 0) or (((y_p-y_0)/(x_p - x_0) > 1) and defined_domain[idx_y-1, idx_x+1] == 0)):
                    
                    thickness = sqrt(dx_data**2+dy_data**2)/2
                        
                    x_wall = (y_0-y_p+x_p+x_0)/2
                    y_wall = (y_0+y_p+x_0-x_p)/2
                        
                    x_boundary_layer = np.array([x_wall - thickness/sqrt(2), y_wall - thickness/sqrt(2)])                        
                        
                    vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
                    y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
                    # Velocity in coordinates parallel to diagonal boundary transformation
                    u_new = (-vel[0]+vel[1])/sqrt(2)
                    v_new = (-vel[0]-vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
                    u = -(u_new+v_new)/sqrt(2)
                    v = (u_new-v_new)/sqrt(2)
                    
                else:
                        
                    return velocity_domain(t, x, Interpolant, dt_data, bool_unsteady)
                    
            elif defined_domain[idx_y-1, idx_x-1] == 0:
                    
                x_0 = X[idx_y, idx_x]
                y_0 = Y[idx_y, idx_x]
            
                p1norm = (abs(y_p-y_0)+ abs(x_p-x_0)) > 1
                    
                if (p1norm and ((y_p-y_0)/(x_p - x_0) >= 1 and defined_domain[idx_y-1, idx_x] == 0) or (((y_p-y_0)/(x_p - x_0) < 1) and defined_domain[idx_y, idx_x-1] == 0)):
                    
                    thickness = sqrt(dx_data**2+dy_data**2)/2
                        
                    x_wall = (y_0-y_p+x_p+x_0)/2
                    y_wall = (y_0+y_p+x_0-x_p)/2
                        
                    x_boundary_layer = np.array([x_wall + thickness/np.sqrt(2), y_wall + thickness/np.sqrt(2)])                        
                        
                    vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
                    y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
                    # Velocity in coordinates parallel to diagonal boundary transformation
                
                    u_new = (-vel[0]+vel[1])/sqrt(2)
                    v_new = (-vel[0]-vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
                    u = -(u_new+v_new)/sqrt(2)
                    v = (u_new-v_new)/sqrt(2)
                    
                else:
                        
                    return velocity_domain(t, x, Interpolant, dt_data, bool_unsteady)                    
        
            elif defined_domain[idx_y-1, idx_x] == 0:
                
                x_0 = X[idx_y, idx_x-1]
                y_0 = Y[idx_y, idx_x-1]
                    
                p1norm = (abs(y_p-y_0)+ abs(x_p-x_0)) < 1
                        
                if (p1norm and (((y_p-y_0)/(x_p - x_0) < -1 and defined_domain[idx_y-2, idx_x-1]) == 0) or (((y_p-y_0)/(x_p - x_0) >= -1) and defined_domain[idx_y, idx_x+1])):
            
                    wall_orientation = np.array([1, 1])
                    thickness = np.sqrt(dx_data**2+dy_data**2)/2
            
                    x_wall = ((y_p-y_0)+x_p+x_0)/2
                    y_wall = ((y_p+y_0)+x_p-x_0)/2
            
                    x_boundary_layer = np.array([x_wall - thickness/sqrt(2), y_wall + thickness/sqrt(2)])
                                    
                    vel = velocity_domain(x_boundary_layer, t)
            
                    # Coordinate transformation 
                    y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
                    u_new = (vel[0]+vel[1])/sqrt(2)
                    v_new = (-vel[0]+vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
                    u = (u_new-v_new)/sqrt(2)
                    v = (u_new+v_new)/sqrt(2)
                    
                else:
                        
                    return velocity_domain(t, x, Interpolant, dt_data, bool_unsteady)
                    
            elif defined_domain[idx_y, idx_x-1] == 0:
                
                x_0 = X[idx_y, idx_x-1]
                y_0 = Y[idx_y, idx_x-1]
                    
                p1norm = (abs(y_p-y_0)+ abs(x_p-x_0)) >= 1
                        
                if (p1norm and (((y_p-y_0)/(x_p - x_0) < -1 and defined_domain[idx_y-2, idx_x-1]) == 0) or (((y_p-y_0)/(x_p - x_0) >= -1) and defined_domain[idx_y, idx_x+1])):
            
                    wall_orientation = np.array([1, 1])
                    thickness = np.sqrt(dx_data**2+dy_data**2)/2
            
                    x_wall = ((y_p-y_0)+x_p+x_0)/2
                    y_wall = ((y_p+y_0)+x_p-x_0)/2
            
                    x_boundary_layer = np.array([x_wall - thickness/sqrt(2), y_wall + thickness/sqrt(2)])
                                    
                    vel = velocity_domain(t, x_boundary_layer, Interpolant, dt_data, bool_unsteady)
            
                    # Coordinate transformation 
                    y_tilde = sqrt((x_wall-x_p)**2+(y_wall-y_p)**2)
            
                    u_new = (vel[0]+vel[1])/sqrt(2)
                    v_new = (-vel[0]+vel[1])/sqrt(2)*(y_tilde/thickness)**2
            
                    u = (u_new-v_new)/sqrt(2)
                    v = (u_new+v_new)/sqrt(2)
                
                else:
                        
                    return velocity_domain(t, x, Interpolant, dt_data, bool_unsteady)
    else:
            
        return None
    
    vel = np.array([u, v])
        
    return vel